# [Advent of Code 2022 Day ?]()

In [1]:
from __future__ import annotations
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2
run_doctest_for = lambda func: doctest.run_docstring_examples(func, globals())

## Test Cases

In [2]:
PART_ONE_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 31,
    },
    "input": {
        "input1": 879,
    },
}
PART_ONE_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}
PART_ONE_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}

In [3]:
PART_TWO_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example2": 3,
    },
    "input": {
        "input1": 539051801941,
    },
}
PART_TWO_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}
PART_TWO_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}

## Input Parsing

In [4]:
class Model(BaseModel):
    data: Any

def parse_input_from_filename(filename: str) -> Context:
    lines = list(yield_line(filename))

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    for idx, line in enumerate(lines):
        input_lines.append(line)

    return ctx

In [5]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["example"].keys() | PART_TWO_CASES["example"].keys())
def test_parsing_examples(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

..                                                                                           [100%]
============================================= PASSES ==============================================
_________________________________ test_parsing_examples[example2] _________________________________
-------------------------------------- Captured stderr call ---------------------------------------
C200B40A82
_________________________________ test_parsing_examples[example1] _________________________________
-------------------------------------- Captured stderr call ---------------------------------------
A0016C880162017C3686B18A3D4780
2 passed in 0.02s


In [6]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["input"].keys() | PART_TWO_CASES["input"].keys())
def test_parsing_inputs(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

.                                                                                            [100%]
============================================= PASSES ==============================================
___________________________________ test_parsing_inputs[input1] ___________________________________
-------------------------------------- Captured stderr call ---------------------------------------
A052E04CFD9DC0249694F0A11EA2044E200E9266766AB004A525F86FFCDF4B25DFC401A20043A11C61838600FC678D51B8C0198910EA1200010B3EEA40246C974EF003331006619C26844200D414859049402D9CDA64BDEF3C4E623331FBCCA3E4DFBBFC79E4004DE96FC3B1EC6DE4298D5A1C8F98E45266745B382040191D0034539682F4E5A0B527FEB018029277C88E0039937D8ACCC6256092004165D36586CC013A008625A2D7394A5B1DE16C0E3004A8035200043220C5B838200EC4B8E315A6CEE6F3C3B9FFB8100994200CC59837108401989D056280803F1EA3C41130047003530004323DC3C860200EC4182F1CA7E452C01744A0A4FF6BBAE6A533BFCD1967A26E20124BE1920A4A6A613315511007A4A32BE9AE6B5CAD19E56BA1430053803341007E24C168A6

## Helper Functions

### Hexchar to Bits

In [7]:
%%ipytest -xrPvvvvv

def hexchar_to_bits(char: str) -> str:
    assert len(char) == 1
    return {
        "0": "0000",
        "1": "0001",
        "2": "0010",
        "3": "0011",
        "4": "0100",
        "5": "0101",
        "6": "0110",
        "7": "0111",
        "8": "1000",
        "9": "1001",
        "A": "1010",
        "B": "1011",
        "C": "1100",
        "D": "1101",
        "E": "1110",
        "F": "1111",
    }[char]

def test_helper_1() -> None:
    assert hexchar_to_bits("0") == "0000"
    assert hexchar_to_bits("F") == "1111"

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.02s


### Hexstring to Bits

In [8]:
%%ipytest -xrPvvvvv

def hex_to_bits(input: str) -> str:
    res = []
    for char in input:
        res.append(hexchar_to_bits(char))
    return "".join(res)

def test_helper_2() -> None:
    assert hex_to_bits("D2FE28") == "110100101111111000101000"
    assert hex_to_bits("38006F45291200") == "00111000000000000110111101000101001010010001001000000000"
    assert hex_to_bits("EE00D40C823060") == "11101110000000001101010000001100100000100011000001100000"

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


### Consume String

In [9]:
%%ipytest -xrPvvvvv

def consume(amount: int, string: str) -> tuple[str, str]:
    assert len(string) >= amount, f"Attempted to consume {amount} chars from string with only length {len(string)}: '{string}'"
    first = string[:amount]
    second = string[amount:]
    assert string == first + second
    return first, second

def test_consume() -> None:
    assert consume(3, "hello") == ("hel", "lo")
    assert consume(5, "boobs") == ("boobs", "")
    with pytest.raises(AssertionError):
        consume(6, "hello")

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.04s


### Get Version and ID

In [10]:
%%ipytest -xrPvvvvv

def get_version_and_id(packet: str) -> tuple[str, str]:
    version_string, packet = consume(3, packet)
    p_version = int(version_string, 2)
    id_string, packet = consume(3, packet)
    p_id = int(id_string, 2)

    return p_version, p_id

def test_get_version_and_id() -> None:
    assert get_version_and_id("110100101111111000101000") == (6, 4)

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.02s


### Interpret First Packet Type

In [11]:
%%ipytest -xrPvvvvv

class PacketType(Enum):
    OPERATOR = 0
    LITERAL = 1

def get_first_type(data: str) -> PacketType:
    pver, pid = get_version_and_id(data)
    if pid == 4:
        return PacketType.LITERAL
    return PacketType.OPERATOR


no tests ran in 0.01s


### Parse Packet

In [12]:
%%ipytest -xrPvvvvv

def parse_packet(packet: str, level: int = 0) -> Any:

    parsed = 0

    pver, pid = get_version_and_id(packet)
    chunk, packet = consume(6, packet)
    parsed += 6

    packet_type = "Literal" if pid == 4 else "Operator"

    data = {
        "version": pver,
        "id": pid,
        "type": packet_type,
    }

    if packet_type == "Literal":
        vals = []
        while True:
            chop, packet = consume(5, packet)
            parsed += 5
            vals.append(chop[1:])
            if not chop[0] == "1":
                break
        data["decval"] = int("".join(vals), 2)
        data["parsed"] = parsed

    elif packet_type == "Operator":
        data["children"] = []
        length_type, packet = consume(1, packet)
        parsed += 1

        if length_type == "0":
            total_size_str, packet = consume(15, packet)
            total_size = int(total_size_str, 2)
            parsed += 15
            subparse = 0
            while subparse < total_size:
                subpacket = parse_packet(packet, level + 1)
                chars_parsed = subpacket["parsed"]
                subpacket_text, packet = consume(chars_parsed, packet)
                subparse += chars_parsed
                data["children"].append(subpacket)
            parsed += subparse
            data["parsed"] = parsed

        elif length_type == "1":
            num_packets_str, packet = consume(11, packet)
            num_packets = int(num_packets_str, 2)
            parsed += 11
            subcount = 0
            subparse = 0
            while subcount < num_packets:
                subpacket = parse_packet(packet, level + 1)
                chars_parsed = subpacket["parsed"]
                subpacket_text, packet = consume(chars_parsed, packet)
                subparse += chars_parsed
                subcount += 1
                data["children"].append(subpacket)
            parsed += subparse
            data["parsed"] = parsed

    return data


no tests ran in 0.01s


In [13]:
%%ipytest -xrPvvvvv

def test_helper_5() -> None:
    assert hex_to_bits("38006F45291200") == "00111000000000000110111101000101001010010001001000000000"
    example1 = parse_packet("00111000000000000110111101000101001010010001001000000000")
    assert example1["children"][0]["decval"] == 10 and example1["children"][1]["decval"] == 20

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


In [14]:
%%ipytest -xrPvvvvv

def test_parse_afewefaadfsdfasdwefawef() -> None:
    example = parse_packet("01010000001")
    assert example["type"] == "Literal"
    assert example["decval"] == 1

def test_parse_afewefawefawef() -> None:
    example = parse_packet("10010000010")
    assert example["type"] == "Literal"
    assert example["decval"] == 2

def test_parse_awefawef() -> None:
    example = parse_packet("0011000001100000")
    assert example["type"] == "Literal"
    assert example["decval"] == 3

def test_parse_example2() -> None:
    assert hex_to_bits("EE00D40C823060") == "11101110000000001101010000001100100000100011000001100000"
    example = parse_packet("11101110000000001101010000001100100000100011000001100000")
    assert example["type"] == "Operator"
    assert example["version"] == 7
    assert example["id"] == 3
    assert len(example["children"]) == 3
    assert example["children"][0]["decval"] == 1
    assert example["children"][1]["decval"] == 2
    assert example["children"][2]["decval"] == 3

....                                                                                         [100%]
============================================= PASSES ==============================================
4 passed in 0.03s


### Get Version Sum

In [15]:
%%ipytest -xrPvvvvv

def get_version_sum_for_hexstring(hexstring: str) -> int:
    bits = hex_to_bits(hexstring)
    parsed = parse_packet(bits)

    final_sum = 0

    def traverse(packet) -> None:
        nonlocal final_sum
        final_sum += packet["version"]
        if "children" in packet:
            for child in packet["children"]:
                traverse(child)

    traverse(parsed)

    return final_sum

@pytest.mark.parametrize(
    "string, expected",
    [
        ("8A004A801A8002F478", 16),
        ("620080001611562C8802118E34", 12),
        ("C0015000016115A2E0802F182340", 23),
        ("A0016C880162017C3686B18A3D4780", 31),
    ]
)
def test_difficult(string: str, expected: int) -> None:
    assert get_version_sum_for_hexstring(string) == expected


....                                                                                         [100%]
============================================= PASSES ==============================================
4 passed in 0.03s


### Solve Packet

In [16]:
%%ipytest -xrPvvvvv

def solve_packet(packet: dict[str, Any]) -> int:
    type_id = packet["id"]

    if packet["type"] == "Literal":
        return packet["decval"]

    ans = 0

    if type_id == 0:
        for child in packet["children"]:
            ans += solve_packet(child)
    elif type_id == 1:
        if not packet["children"]:
            return 0  # ???
        ans = 1
        for child in packet["children"]:
            ans *= solve_packet(child)
    elif type_id == 2:
        return min([solve_packet(child) for child in packet["children"]])
    elif type_id == 3:
        return max([solve_packet(child) for child in packet["children"]])
    elif type_id == 5:
        assert len(packet["children"]) == 2, f"GT packet needs exactly 2 children but encountered {len(packet['children'])} -> {packet['children']}"
        return 1 if solve_packet(packet["children"][0]) > solve_packet(packet["children"][1]) else 0
    elif type_id == 6:
        assert len(packet["children"]) == 2, f"LT packet needs exactly 2 children but encountered {len(packet['children'])} -> {packet['children']}"
        return 1 if solve_packet(packet["children"][0]) < solve_packet(packet["children"][1]) else 0
    elif type_id == 7:
        assert len(packet["children"]) == 2, f"EQ packet needs exactly 2 children but encountered {len(packet['children'])} -> {packet['children']}"
        return 1 if solve_packet(packet["children"][0]) == solve_packet(packet["children"][1]) else 0
    else:
        raise ValueError(f"Invalid {type_id=}")

    return ans


no tests ran in 0.01s


### Evaluate Packet String

In [17]:
%%ipytest -xrPvvvvv

def evaluate_packet(data: str) -> int:
    bits = hex_to_bits(data)
    parsed = parse_packet(bits)

    return solve_packet(parsed)

@pytest.mark.parametrize(
    "packetstr, expected",
    [
        ("C200B40A82", 3),
        ("04005AC33890", 54),
        ("880086C3E88112", 7),
        ("CE00C43D881120", 9),
        ("D8005AC2A8F0", 1),
        ("F600BC2D8F", 0),
        ("9C005AC2F8F0", 0),
        ("9C0141080250320F1802104A08", 1),
    ]
)
def test_evaluate_packet(packetstr: str, expected: int) -> None:
    assert evaluate_packet(packetstr) == expected

........                                                                                     [100%]
============================================= PASSES ==============================================
8 passed in 0.04s


## Main Function

In [18]:
def solve(part: int, filename: str) -> int:
    input = parse_input_from_filename(filename).input
    if part == 1:
        return get_version_sum_for_hexstring(input[0])
    if part == 2:
        return evaluate_packet(input[0])
    else:
        raise Exception(f"Invalid part: {part}")

### Part 1

In [19]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["example"].items())
def test_part_one_examples(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["input"].items())
def test_part_one_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 0.02s


### Part 2

In [20]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["example"].items())
def test_part_two_examples(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["input"].items())
def test_part_two_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 0.02s


dadada